### Test LLM Connection

In [1]:
from gai.lib.config import config_helper
from gai.llm.openai import AsyncOpenAI

# "llama3.2:3b" for Ollama with CPU
# or "ttt" for Gai/Exl2 running llama3.1-exl2 with GPU
# or "sonnet-4" for Claude
# or "gpt-4o" for OpenAI

model_name = "llama3.2:3b"
llm_config = config_helper.get_client_config(model_name)

# Start streaming output

client = AsyncOpenAI(client_config=llm_config)
response = await client.chat.completions.create(
    model=model_name, 
    messages=[{"role": "user", "content": "tell me a one sentence story"}],
    stream=True,
)
async for chunk in response:
    if chunk:
        print(chunk.extract(), end="", flush=True)


As she opened the old, mysterious box in her attic, Emily discovered a faded photograph of herself as a child, standing alongside a stranger who looked uncannily like her own future husband.

### Test Single-Agent Session

In [4]:
from gai.asm.agents import ToolUseAgent
from gai.mcp.client.mcp_client import McpAggregatedClient
from gai.messages.monologue import FileMonologue
from gai.messages import message_helper

model_name = "sonnet-4"
llm_config = config_helper.get_client_config(model_name)
monologue = FileMonologue(file_path="/tmp/monologue.txt")
agent = ToolUseAgent(
    agent_name="ToolUseAgent",
    llm_config=llm_config,
    aggregated_client=McpAggregatedClient(["mcp-time"]),
    monologue=monologue,
)

print("Round 1")

user_message = "What is the time in Singapore now?"

print("Turn 1:")

resp = agent.start(user_message=user_message)
async for chunk in resp:
    message_helper.print_chunk(chunk)
    
print("Turn 2:")

resp = agent.resume()
async for chunk in resp:
    message_helper.print_chunk(chunk)


Round 1
Turn 1:
I'll get the current time in Singapore for you.
Tool: "current_time"
	Input: format = YYYY-MM-DD HH:mm:ss
	Input: timezone = Asia/Singapore
Turn 2:
The current time in Singapore is **7:59:51 PM on August 5, 2025** (19:59:51 in 24-hour format).


In [5]:
print("Round 2")

agent = ToolUseAgent(
    agent_name="ToolUseAgent",
    llm_config=llm_config,
    aggregated_client=McpAggregatedClient(["mcp-web"]),
    monologue=monologue,
)
user_message = "What is the top 10 news highlight from asiaone.com?"

print("Turn 1:")

resp = agent.resume(user_message=user_message)
async for chunk in resp:
    message_helper.print_chunk(chunk)

print("Turn 2:")

resp = agent.resume()
async for chunk in resp:
    message_helper.print_chunk(chunk)


Round 2
Turn 1:
I'll help you get the top 10 news highlights from AsiaOne.com. Let me scrape their website to gather the current news headlines.
Tool: "scrape"
	Input: url = https://www.asiaone.com
Turn 2:
Based on the content from AsiaOne.com, here are the **top 10 news highlights** currently featured on their website:

## Top 10 News Highlights from AsiaOne.com

1. **Malaysian authorities searching for Singapore driver who filled container with subsidised fuel** - A cross-border fuel incident involving Singapore drivers

2. **'I tried to save her': Friend weeps beside coffin of 23-year-old woman killed in Yishun car crash** - Tragic traffic accident in Singapore with emotional aftermath

3. **5 durians for $488: Punggol residents accuse door-to-door salesman of selling them sour, unripe fruits** - Consumer complaints about overpriced, poor-quality durians

4. **Wife of man who died in Bedok North accident looking for good Samaritan who administered CPR** - Community search for helpfu

In [11]:
import json
messages = monologue.list_chat_messages()
print(json.dumps(messages,indent=2))

[
  {
    "role": "user",
    "content": "\n            Your name is ToolUseAgent within the context of this conversation and you will always respond as such.\n            Do not refer to yourself as an AI or a bot or confuse your name with other agents.\n           \n            You may respond to my following message using the context you have learnt.\n            What is the time in Singapore now?\n            \n            You may ask me for more information if you need to clarify my request but ask just enough to get the information you need to get started.\n            If you need to ask for more information, please use the \"user_input\" tool to get the information from me.\n            Never call \"user_input\" without providing a description of what you need from me.\n            Do not be vague and expect an input from me, be specific about what you want.\n            "
  },
  {
    "role": "assistant",
    "content": [
      {
        "citations": null,
        "text": "I'll


### Multi-Agent Session

In [ ]:

import os
from gai.lib.constants import DEFAULT_GUID
from gai.sessions import SessionManager
from rich.console import Console
from gai.nodes.agent_node import AgentNode
from gai.nodes.user_node import UserNode
from dotenv import load_dotenv

load_dotenv()
console = Console(force_terminal=True)

os.environ["LOG_LEVEL"] = "Warning"

# Initiate a session

session_mgr = SessionManager(
    dialogue_id=DEFAULT_GUID, file_path=os.path.join("tmp", "dialogue.json")
)
session_mgr.reset()
await session_mgr.start()

## Create plan

flow_plan = """
    User ->> HaikuWriter
    HaikuWriter ->> HaikuReviewer
    """

## Register HaikuiWriter and wait for User

HaikuWriter = AgentNode(
    agent_name="HaikuWriter", model_name="llama3.2:3b", session_mgr=session_mgr
)
await HaikuWriter.subscribe(flow_plan)

## Register Diana and wait for User

HaikuReviewer = AgentNode(
    agent_name="HaikuReviewer", model_name="ttt", session_mgr=session_mgr
)
await HaikuReviewer.subscribe(flow_plan)

user = UserNode(session_mgr=session_mgr)

## START Chain Response
resp = await user.start(
    user_message="You will work as a team to write a haiku poem about the beauty of coding and review it. Please share your thoughts while you are writing. Do not ask for any input from me.",
    flow_plan=flow_plan,
)
content = ""
async for chunk in resp:
    if not content:
        console.print(f"[bright_green]{chunk}[/bright_green] :")
        content = chunk
    else:
        print(chunk, end="", flush=True)
        content += chunk

## RESUME Chain Response
resp = await user.resume()
content = ""
async for chunk in resp:
    if not content:
        console.print(f"[bright_green]{chunk}[/bright_green] :")
        content = chunk
    else:
        print(chunk, end="", flush=True)
        content += chunk


HaikuWriter :

I'd like to clarify that I'll be working with another team member to co-write this haiku poem. What would you like us to focus on in terms of the beauty of coding - is it the creative problem-solving, the joy of building something from scratch, or perhaps the sense of community and collaboration that comes with coding?

HaikuReviewer :

           It seems like we're ready to begin crafting our haiku. Let's focus on the joy of building something from scratch, as it encompasses the excitement and fulfillment that comes with seeing a project take shape before one's eyes. The process of coding allows developers to bring their ideas to life, transforming abstract concepts into tangible solutions. 

The first line of our haiku could revolve around this idea of creation. How about something like: "In lines of code, a vision takes form"? Do you think this sets the right tone for our poem?
